In [ ]:
import numpy as np
import torch
import torch.nn as nn
from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.data.lstm_preprocessor import preprocess_data

# Define constants for input size, sequence length, etc.
INPUT_SIZE = 153
HIDDEN_SIZE = 20
NUM_LAYERS = 2
SEQ_LENGTH = 24
BATCH_SIZE = 32
LEARNING_RATE = 0.01
NUM_EPOCHS = 100

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 3)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


X, y = get_data_with_features_and_labels('BTC', '1h', (0.02, 0.005))
X_train, y_train, X_val, y_val = preprocess_data(X, y, lookback=SEQ_LENGTH, test_pct=0.3)

# Create the training data
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train.values).long()

# Initialize the model and set device to GPU if present
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
for epoch in range(NUM_EPOCHS):
    # Forward pass
    outputs = model(X_train.to(device))
    loss = criterion(outputs, y_train.to(device))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, NUM_EPOCHS, loss.item()))


In [1]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.data.lstm_preprocessor import preprocess_data
from torch.utils.data import TensorDataset, DataLoader
# from vctr.models.lstm.main import LSTM_Model

TIMESTEPS = 3
FEATURES = 153
BATCH_SIZE = 256
CLASSES = 3
LABEL_ARGS = (0.02, 0.005)
TIMEFRAME = '1H'
SYMBOL = 'BTC'

X, y = get_data_with_features_and_labels(SYMBOL, TIMEFRAME, LABEL_ARGS)
X_train, y_train, X_val, y_val = preprocess_data(X, y, lookback=TIMESTEPS, test_pct=0.4)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train.values).long()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val.values).long()

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


def train(model, X_train, y_train, X_val, y_val, learning_rate, num_epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_accuracy = 0.0

        for inputs, labels in zip(X_train, y_train):
            # inputs = torch.tensor(inputs).unsqueeze(0).float()
            # labels = torch.tensor([labels]).long()

            # Assuming that labels is currently a list of integers
            # labels_tensor = labels
            # labels = labels_tensor.long()

            optimizer.zero_grad()

            outputs = model(inputs)
            print('==>> outputs: ', outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_accuracy += (predicted == labels).sum().item() / len(labels)

        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0

        with torch.no_grad():
            for inputs, labels in zip(X_val, y_val):
                inputs = torch.tensor(inputs).unsqueeze(0).float()
                labels = torch.tensor([labels]).long()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_accuracy += (predicted == labels).sum().item() / len(labels)

        print(
            f'Epoch [{epoch+1}/{num_epochs}], '
            f'Train Loss: {train_loss/len(y_train):.4f}, Train Accuracy: {train_accuracy/len(y_train)*100:.2f}%, '
            f'Val Loss: {val_loss/len(y_val):.4f}, Val Accuracy: {val_accuracy/len(y_val)*100:.2f}%'
        )


In [2]:
model = LSTM_Model('lstm_1', TIMESTEPS, BATCH_SIZE, FEATURES, n_classes=CLASSES)
train(model, X_train, y_train, X_val, y_val, 0.001, 10)

AttributeError: 'tuple' object has no attribute 'dim'